In [ ]:
import os
import geopandas
import pycountry
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from datetime import datetime, timedelta

In [ ]:
def alpha3_to_alpha2(code):
    country = pycountry.countries.get(alpha_3=code)
    if country is not None:
        return country.alpha_2
    else:
        return None

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
world['alpha2'] = world.apply(lambda row: alpha3_to_alpha2(row.iso_a3), axis=1)
world = world.drop(['pop_est', 'continent', 'gdp_md_est', 'iso_a3'], axis=1)
world = world[(world.name != "Antarctica")]
world = world.to_crs(epsg=3395)
world.head()

In [ ]:
anomalies = pd.read_csv('anomalies_final.csv')
anomalies.date = pd.to_datetime(anomalies.date)
anomalies.score = anomalies.score.abs()
anomalies.head()

In [ ]:
start_date = datetime.strptime('2020-01-01', '%Y-%m-%d')
end_date = datetime.now()

In [ ]:
def get_score(region):
    score = df[df.region==region].score
    if len(score)==0:
        return 0
    return np.array(score)[0]

In [ ]:
cmap = mcolors.LinearSegmentedColormap.from_list("", ["limegreen","darkorange","firebrick"])

df = anomalies[anomalies.date>=start]
df = df[df.date<end]
df = df.groupby(df.region).sum().reset_index()
world['score'] = world.apply(lambda row: get_score(row.alpha2), axis=1)
world.plot(column=world.score, 
           cmap=cmap, legend=True,
           missing_kwds={
               "color": "lightgrey",
               "edgecolor": "red",
               "hatch": "/",
               "label": "Missing values",
           })